# Generate Technical Indicators
---

### Import Libraries

In [14]:
import os
import requests 
import pandas_datareader as pdr

import functools
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns
import warnings

from sklearn import preprocessing 

from importlib import reload
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227

### Import Data

In [93]:
ticker = 'TSLA'

In [94]:
stock_df = pdr.get_data_yahoo(ticker, '1980')
# stock_df.to_csv('data/'+csv_title+'.csv')
stock_df.to_csv('data/raw_stocks/'+ticker.upper()+'.csv')

In [95]:
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-22,997.859985,921.750000,930.000000,993.979980,35289500.0,993.979980
2022-03-23,1040.699951,976.400024,979.940002,999.109985,40225400.0,999.109985
2022-03-24,1024.489990,988.799988,1009.729980,1013.919983,22973600.0,1013.919983
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900.0,1010.640015
2022-03-28,1097.879883,1053.599976,1065.099976,1091.839966,33666903.0,1091.839966


In [96]:
stock_df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,2958.000000,2958.000000,2958.000000,2958.000000,2.958000e+03,2958.000000
mean,142.392339,136.027734,139.298368,139.379139,3.131170e+07,139.379139
std,256.889825,244.788901,251.048930,251.163575,2.797508e+07,251.163575
min,3.326000,2.996000,3.228000,3.160000,5.925000e+05,3.160000
25%,20.504000,19.197500,19.673500,19.704500,1.310850e+07,19.704500
50%,47.493000,45.825001,46.688000,46.569000,2.488680e+07,46.569000
75%,69.399500,66.948997,68.088501,68.200003,3.973162e+07,68.200003
max,1243.489990,1217.000000,1234.410034,1229.910034,3.046940e+08,1229.910034


### Check for missing data

In [97]:
print('No Missing Data') if sum(stock_df.isna().sum())==0 else stock_df.isna().sum()

No Missing Data


---
# Technical Indicator Calculations  
---

### Momentum

In [98]:
def momentum(close, days):
    m = [None for i in range(days)]
    for i in range(len(close) - days):
        end = i + days
        m.append(close[i] - days)
    return m 

### Relative Strength Index (RSI)

In [99]:
def gain(x):
    if x > 0:
        return x
    else:
        return 0 
        
def loss(x):
    if x < 0:
        return abs(x)
    else:
        return 0 

    
def rsi(stock):
    rsi_list = [None for i in range(14)]
    stock = stock.Change
    
    # Calculate Relative Strength (RS) for first 15 
    avg_gain = sum([i for i in stock[1:15] if i > 0])/14
    avg_loss = sum([abs(i) for i in stock[1:15] if i < 0])/14 
    rs = avg_gain/avg_loss
    # Calculate Relative Strength Index (RSI) for first 15
    rsi = 100 - (100/(1+rs))
    rsi_list.append(rsi)
    
    # Calculate Relative Strength(RS) for 15+ 
    for i in range(15, len(stock)):
        avg_gain = (avg_gain * 13 + gain(stock[i]))/14
        avg_loss = (avg_loss * 13 + loss(stock[i]))/14
        rs = avg_gain/avg_loss
        
        # Calculate Relative Strength Index (RSI) for 15+ 
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    
    return rsi_list

### Moving Average Convergence/Divergence (MACD)

In [100]:
def macd(stock):
    exp1 = stock.Close.ewm(span=12, adjust=False).mean()
    exp2 = stock.Close.ewm(span=26, adjust=False).mean()
    ma_con_div = exp1-exp2
    signal = ma_con_div.ewm(span=9, adjust=False).mean()
    return ma_con_div, signal

### Bollinger Bands

In [101]:
def bollinger(stock, window=21):
    roll_mean = stock.Close.rolling(window).mean()
    roll_std = stock.Close.rolling(window).std()
    upper_band = roll_mean + (roll_std*2)
    lower_band = roll_mean - (roll_std*2)
    return upper_band, lower_band

---
# Generating Technical Indicators 
---

In [102]:
# gets list of all files and directories
# git add .ipynb_checkpoints directory -> causes issues 
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    if file.endswith('.csv'):
        name = file.lower().split(".")[0]
        stocks[name] = pd.read_csv('data/raw_stocks/'+file)
    
        # Date Feature
        stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
        stocks[name].set_index('Date', inplace = True)
        # Return Feature 
        stocks[name]['Return'] = round(stocks[name]['Close']/stocks[name]['Open']-1, 3)
        # Change Feature: Change previous day's price by absolute value
        stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
        # Volatility Feature
        stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
        # Moving Average: 7 Days
        stocks[name]['MA7'] = stocks[name].Close.rolling(window=7).mean()
        # Moving Average: 21 Days
        stocks[name]['MA21'] = stocks[name].Close.rolling(window=21).mean()
    
        # Momentum
        stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
        # Relative Strength Index (RSI)
        stocks[name]['RSI'] = rsi(stocks[name])
        # Moving Average Convergence/Divergence (MACD)
        stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
        # Bollinger Bands
        stocks[name]['Upper_Band'], stocks[name]['Lower_Band'] = bollinger(stocks[name])
    
        # Saving
        # stocks[name].dropna(inplace=True)
        stocks[name].to_csv('data/stocks/'+name+'.csv')

In [103]:
stocks[ticker.lower()].tail()

,High,Low,Open,Close,Volume,Adj Close,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_Band,Lower_Band
Date,,,,,,,,,,,,,,,,,
2022-03-22,997.859985,921.750000,930.000000,993.979980,35289500.0,993.979980,0.069,72.820007,87.439689,871.517133,843.366661,868.599976,64.389207,5.136793,-15.888703,950.822394,735.910929
2022-03-23,1040.699951,976.400024,979.940002,999.109985,40225400.0,999.109985,0.020,5.130005,88.507920,904.765703,851.822850,902.390015,64.816114,14.527562,-9.805450,978.322784,725.322916
2022-03-24,1024.489990,988.799988,1009.729980,1013.919983,22973600.0,1013.919983,0.004,14.809998,90.072894,935.055699,863.721898,918.159973,66.080344,22.900870,-3.264186,1002.002396,725.441400
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900.0,1010.640015,0.003,-3.279968,91.052952,959.399990,873.715707,990.979980,65.518846,28.938525,3.176356,1022.800967,724.630447
2022-03-28,1097.879883,1053.599976,1065.099976,1091.839966,33666903.0,1091.839966,0.025,81.199951,97.304088,990.862845,887.142848,996.109985,71.887475,39.816587,10.504402,1060.836798,713.448899
